# Mathematical Engineering - Financial Engineering, FY 2024-2025
<hr>

# Risk Management - Exercise 2: Market Implied vs Historical Default Probabilities

In [1]:
# Importing relevant libraries. The bootstrap_py package has been built in the previous assignment.
# The utilities module contains some useful functions.

from scipy.optimize import fsolve

from utilities.ex1_utilities import business_date_offset, year_frac_act_x
from utilities.ex2_utilities import (
    defaultable_bond_dirty_price_from_intensity,
    defaultable_bond_dirty_price_from_z_spread,
)

from bootstrap_py import Bootstrap

import pandas as pd
import numpy as np

First, we run the bootstrap on the Market Data, using the `Bootstrap.fit()` function. In the following, it will be convient to treat `discount_factors` as a `pd.Series`, with the relevant dates as index:

In [2]:
b = Bootstrap.from_xls("MktData_CurveBootstrap.xls")

dates, discount_factors = b.fit()
discount_factors = pd.Series(discount_factors, index=dates)

today = pd.Timestamp("2023-02-02")

Our portofolo is made up of two corporate bonds issued by the same company, the first with $1$ year maturity and the second with $2$ years maturity. Other details on the contracts are defined in the cell below

In [3]:
# Parameters
maturity1 = 1  # Maturity in years
maturity2 = 2 # ERROR MATURITY IS TWO YEARS
notional1 = 1e7
notional2 = 1e7
coupon_rate1 = 0.05
coupon_rate2 = 0.06
coupon_freq1 = 2  # Coupon frequency in payments a years
coupon_freq2 = 2
dirty_price1 = 100
dirty_price2 = 102

rating = "IG"  # Credit rating

# Expiries are a pd.Timestamp object, using business_date_offset 
# we guarantee they're business days
expiry1 = business_date_offset(today, year_offset=maturity1)
expiry2 = business_date_offset(today, year_offset=maturity2)

### 1. Intensity Based Model: constant intensity

We calibrate the intesity model to the market data, under the assumption that the intensity $\lambda$ is constant. The recovery rate is set to $\pi = 30\%$. Calibratrion is done by numerically solving the equation 

$$
P(\lambda \vert t,T) = \hat{P}(t,T)
$$

where $P(\lambda \vert t,T)$ is the price of the defaultable bond according to the intensity model while $\hat{P}$ is the actual *mid* price quoted on the market.

In [4]:
# Q1: Derive the average intensity for the two bonds
recovery_rate = 0.3

h_1y = fsolve(
    lambda intensity: defaultable_bond_dirty_price_from_intensity(
        today,
        expiry1,
        coupon_rate1,
        coupon_freq1,
        recovery_rate,
        intensity[0],
        #df,
        discount_factors,
        100,
    )
    - dirty_price1,
    x0=0.02,
)[0]

h_2y = fsolve(
    lambda intensity: defaultable_bond_dirty_price_from_intensity(
        today,
        expiry2,
        coupon_rate2,
        coupon_freq2,
        recovery_rate,
        intensity[0],
        #df,
        discount_factors,
        100,
    )
    - dirty_price2,
    x0=0.01,
)[0]

print(f"Average intensity over {maturity1}y: {h_1y:.5%}")
print(f"Average intensity over {maturity2}y: {h_2y:.5%}")
print(f"Mean: {(h_1y + h_2y) / 2:.5%}")

Average intensity over 1y: 2.43896%
Average intensity over 2y: 2.43377%
Mean: 2.43637%


As expected, the implied intesities are very close to each other. Intesities are a measure of the default probability **per unit of time**, so it makes perfectly sense for the market to price equal intensity for two bond issued by the same company.

### 2. Probability of default

Based on the previous calculation, we compute the default probabilities for $1$ and $2$ years. Recall that the survival probability is given by

$$
Prob(t,T) = \exp\Bigg\{-\int_t^T \lambda(s)ds\Bigg\} = \exp\Big\{-\lambda(T-t)\Big\}
$$

In [5]:
# Q2: Default probability estimates
# Survival probabilities

# we take the mean of the two, since they aint exactly the same
intesity = (h_1y+h_2y)/2

surv_prob_1y = np.exp(-intesity*year_frac_act_x(today, expiry1, 365))
surv_prob_2y = np.exp(-intesity*year_frac_act_x(today, expiry2, 365))

# Defaul probabilities
default_prob_1y = 1-surv_prob_1y
default_prob_2y = 1-surv_prob_2y

print(f"{maturity1}y default probability: {default_prob_1y:.5%}")
print(f"{maturity2}y default probability: {default_prob_2y:.5%}")

1y default probability: 2.40693%
2y default probability: 4.76863%


### 3. Z-spreads

Numerically inverting the `defaultable_bond_dirty_price_from_z_spread()` method, we compute the $Z$-spreads.

In [6]:
# Q3: Z-spread calculation
z_spread_1y = fsolve(
    lambda z_spread: defaultable_bond_dirty_price_from_z_spread(
        today,
        expiry1,
        coupon_rate1,
        coupon_freq1,
        z_spread[0],
        discount_factors,
        100,
    )
    - dirty_price1,
    x0=0.02,
)[0]

z_spread_2y = fsolve(
    lambda z_spread: defaultable_bond_dirty_price_from_z_spread(
        today,
        expiry2,
        coupon_rate2,
        coupon_freq2,
        z_spread[0],
        discount_factors,
        100,
    )
    - dirty_price2,
    x0=0.02,
)[0]

print(f"Z-spread over {maturity1}y: {z_spread_1y:0.05f}")
print(f"Z-spread over {maturity2}y: {z_spread_2y:0.05f}")

Z-spread over 1y: 0.01721
Z-spread over 2y: 0.01727


The $Z$-spreads are indeed smaller than the corresponding intensities, in accordance with the theory.

### 4. Intensity Based Model: non-constant intensity

We now assume the intesity $\lambda$ is no longer a constant quantity, but rather piece-wise defined

$$
\lambda(t) = 
\begin{cases}
\lambda_1 \;\; 0 \leq t \leq T_1 \\
\lambda_2 \;\; T_1 \leq t \leq T_2
\end{cases}
$$

where $T_1,T_2$ are the two bonds maturities.

To calibrate the model, we first compute $\lambda_1$, solving the very same equation as we did before. Then, to find $\lambda_2$ we pass use the `fsolve()` method on the second argument of the intensity vector. The pricing function has been modified to handle non-constant intensities in the form of a `pd.Series`.

In [7]:
# Q4: Default probability estimates under the hp. of piecewise constant intensity

h_1y = fsolve(
    lambda intensity: defaultable_bond_dirty_price_from_intensity(
        today,
        expiry1,
        coupon_rate1,
        coupon_freq1,
        recovery_rate,
        pd.Series([intensity[0]], index=[expiry1]),
        discount_factors,
        100,
    )
    - dirty_price1,
    x0=0.02,
)[0]

h_1y2y = fsolve(
    lambda intensity: defaultable_bond_dirty_price_from_intensity(
        today,
        expiry2,
        coupon_rate2,
        coupon_freq2,
        recovery_rate,
        pd.Series([h_1y, intensity[0]], index=[expiry1, expiry2]),
        discount_factors,
        100,
    )
    - dirty_price2,
    x0=0.02,
)[0]

print(f"h_1y: {h_1y:.5%}")
print(f"h_1y2y: {h_1y2y:.5%}")

h_1y: 2.43896%
h_1y2y: 2.42823%


Based on the calibrated intesities, we can calculate the survival probabilities using the same expression as above.

In [8]:
surv_prob_1y = np.exp(-h_1y*year_frac_act_x(today, expiry1, 365))
surv_prob_2y = np.exp(-(h_1y*year_frac_act_x(today, expiry1, 365) + h_1y2y*year_frac_act_x(expiry1, expiry2, 365)))


default_prob_1y = 1-surv_prob_1y
default_prob_2y = 1-surv_prob_2y

print(f"{maturity1}y default probability: {default_prob_1y:.5%}")
print(f"{maturity2}y default probability: {default_prob_2y:.5%}")

1y default probability: 2.40946%
2y default probability: 4.76332%


### 5. Transition Matrix

We now introduce a different model to study the default probabilities on the company, based on the *Transition Matrix*. The company credit worthiness can be in three different states: Investment Grade $IG$, High Yield $HY$ and Default $DEF$. The probabilitie to transition between one class to the other is described by a Markov Chain $A$, represented below

In [9]:
# Q5:Real world default probability from the rating transition matrix
# Simplified rating transition matrix at 1y

transition_matrix = pd.DataFrame(
    [[0.73, 0.25, 0.02], [0.35, 0.6, 0.05], [0, 0, 1]],
    index=["IG", "HY", "Def"],
    columns=["IG", "HY", "Def"],
)
transition_matrix

,IG,HY,Def
IG,0.73,0.25,0.02
HY,0.35,0.60,0.05
Def,0.00,0.00,1.00


We are interested in the default probability for the first and the second year. Since we know that at issue date $t=0$ the company is in state $IG$, the probability of default in one year it's given by its transition probability $\mathbb{P}(IG \to DEF)$. Due to the Markov property, the transitions probability for the second year are just given by the matrxi $A^2$ 

In [10]:
# Q5:Real world default probability from the rating transition matrix
# Transitions probability for the second year

transition_matrix_2y = transition_matrix @ transition_matrix
transition_matrix_2y

,IG,HY,Def
IG,0.6204,0.3325,0.0471
HY,0.4655,0.4475,0.0870
Def,0.0000,0.0000,1.0000


In [11]:
# Q5:Real world default probability from the rating transition matrix
# Default probabilities

print(
    f"One year real world default probability: {transition_matrix.at[rating, 'Def']:.2%}"
)
print(
    f"Two year real world default probability: {transition_matrix_2y.at[rating, 'Def']:.2%}"
)

One year real world default probability: 2.00%
Two year real world default probability: 4.71%


### 6. Scenario 1: Worsening of the credit worthiness

Suppose the credit worthiness of the company drops instantanously, so that the price of the $2$ year bond drops to $97$. We execute a new intensity bootstrap to reconstruct a new $\lambda$. 

In [12]:
# Q6: Estimate the default probabilities under a shock scenario of the mid-term survival probability (Scenario1)
# Computing the new intensities

dirty_price1_shock = dirty_price1
dirty_price2_shock = 97.0

h_1y_shock = fsolve(
    lambda intensity: defaultable_bond_dirty_price_from_intensity(
        today,
        expiry1,
        coupon_rate1,
        coupon_freq1,
        recovery_rate,
        intensity[0],
        discount_factors,
        100,
    )
    - dirty_price1_shock,
    x0=0.02,
)[0]

h_1y2y_shock = fsolve(
    lambda intensity: defaultable_bond_dirty_price_from_intensity(
        today,
        expiry2,
        coupon_rate2,
        coupon_freq2,
        recovery_rate,
        pd.Series([h_1y_shock, intensity[0]], index=[expiry1, expiry2]),
        discount_factors,
        100,
    )
    - dirty_price2_shock,
    x0=0.02,
)[0]

print(f"h_1y2y: {h_1y2y_shock:.5%}")

h_1y2y: 10.20428%


Based on the computed intensities, we re-evalute the survival probabilites

In [13]:
# Q6: Estimate the default probabilities under a shock scenario of the mid-term survival probability (Scenario1)
# Survival probabilities
surv_prob_1y_shock = np.exp(-h_1y_shock*year_frac_act_x(today, expiry1, 365))
surv_prob_2y_shock = np.exp(-(h_1y_shock*year_frac_act_x(today, expiry1, 365)+h_1y2y_shock*year_frac_act_x(expiry1, expiry2, 365)))

# Defaul probabilities
default_prob_1y_shock = 1-surv_prob_1y_shock
default_prob_2y_shock = 1-surv_prob_2y_shock

print(f"{maturity1}y default probability: {default_prob_1y_shock:0.05%}")
print(f"{maturity2}y default probability: {default_prob_2y_shock:0.05%}")

1y default probability: 2.40946%
2y default probability: 11.92589%


### 7. Scenario 2: Improvement in credit worthiness

In this second scenario, we suppose the credit worthiness of the company increases, so that the prices of the two bond quoted on the market both go up: the $1$ year bond is now worth $101$, while the $2$ year bond is worth $103$.

In [14]:
# Q7: Estimate the default probabilities under a shock scenario on overall creditworthiness (Scenario2)
# Computing the new intensities

dirty_price1_shock2 = 101.0
dirty_price2_shock2 = 103.0

h_1y_shock2 = fsolve(
    lambda intensity: defaultable_bond_dirty_price_from_intensity(
        today,
        expiry1,
        coupon_rate1,
        coupon_freq1,
        recovery_rate,
        intensity[0],
        discount_factors,
        100,
    )
    - dirty_price1_shock2,
    x0=0.02,
)[0]

h_1y2y_shock2 = fsolve(
    lambda intensity: defaultable_bond_dirty_price_from_intensity(
        today,
        expiry2,
        coupon_rate2,
        coupon_freq2,
        recovery_rate,
        pd.Series([h_1y_shock2, intensity[0]], index=[expiry1, expiry2]),
        discount_factors,
        100,
    )
    - dirty_price2_shock2,
    x0=0.02,
)[0]


print(f"h_1y: {h_1y_shock2:.5%}")
print(f"h_1y2y: {h_1y2y_shock2:.5%}")

h_1y: 1.00955%
h_1y2y: 2.46549%


In [15]:
# Q7: Estimate the default probabilities under a shock scenario on overall creditworthiness (Scenario2)
# Survival probabilities

surv_prob_1y_shock2 = np.exp(-h_1y_shock2*year_frac_act_x(today, expiry1, 365))
surv_prob_2y_shock2 = np.exp(-(h_1y_shock2*year_frac_act_x(today, expiry1, 365)+h_1y2y_shock2*year_frac_act_x(expiry1, expiry2, 365)))

# Defaul probabilities
default_prob_1y_shock2 = 1-surv_prob_1y_shock2
default_prob_2y_shock2 = 1-surv_prob_2y_shock2

print(f"{maturity1}y default probability: {default_prob_1y_shock2:.2%}")
print(f"{maturity2}y default probability: {default_prob_2y_shock2:.2%}")

1y default probability: 1.00%
2y default probability: 3.43%


### 8. Historical vs Implied

We calculated the analytical conditional probability using the exponential decay relationship between the survival probability and the derived intensity under Scenario 1 (point 6). Default probability is computed as 1 minus the survival probability.

In [26]:
# Q8: Are the conditional default probabilities between the first and the second year derived under the scenario 1
# consistent with the equivalent real-world probabilities derived from the transition matrix?

p_market = np.exp(-h_1y2y_shock*year_frac_act_x(expiry1, expiry2, 365))
p_hist = transition_matrix.at["HY", "Def"]
print(f"Market Implied: {1-p_market:0.05%}")
print(f"Historical: {p_hist:0.05%}")

Market Implied: 9.75139%
Historical: 5.00000%
